PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões

Classificador kNN com seleção de k por validação cruzada, e com cálculo do erro empírico na base de teste HouseholdIncome.


In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import numpy as np

In [ ]:
HHI = pd.read_csv("../input/train.csv", sep=r'\s*,\s*',
        engine='python',
        na_values="")

In [ ]:
HHI

Primeiro, é necessário retirar as colunas que possuem muitos dados faltantes, pois se tentarmos direto retirar as linhas com dados faltantes teremos poucos dados para criar um bom modelo. Além disso, são retiradas as colunas com danos que não são numéricos e aquelas que cujo significado não deve influenciar na predição.

In [ ]:
nHHI = HHI.drop(columns = "v2a1")
nHHI = nHHI.drop(columns ="v18q1")
nHHI = nHHI.drop(columns ="rez_esc")
nHHI = nHHI.drop(columns = "Id")
nHHI = nHHI.drop(columns ="idhogar")
nHHI = nHHI.drop(columns ="dependency")
nHHI = nHHI.drop(columns = "edjefe")
nHHI = nHHI.drop(columns ="edjefa")
nHHI = nHHI.drop(columns ="SQBmeaned")
nHHI = nHHI.drop(columns ="meaneduc")

nHHI.shape

In [ ]:
nHHI.isnull().sum().sum()

Em seguida é feito o mesmo procedimento para a base de teste

In [ ]:
testHHI = pd.read_csv("../input/test.csv", sep=r'\s*,\s*',
        engine='python',
        na_values="")
testHHI

In [ ]:
testHHI.isnull().sum()

In [ ]:
ntestHHI = testHHI.drop(columns = "v2a1")
ntestHHI = ntestHHI.drop(columns ="v18q1")
ntestHHI = ntestHHI.drop(columns ="rez_esc")
ntestHHI = ntestHHI.drop(columns ="Id")
ntestHHI = ntestHHI.drop(columns ="idhogar")
ntestHHI = ntestHHI.drop(columns ="dependency")
ntestHHI = ntestHHI.drop(columns ="edjefe")
ntestHHI = ntestHHI.drop(columns ="edjefa")
ntestHHI = ntestHHI.drop(columns ="SQBmeaned")
ntestHHI = ntestHHI.drop(columns ="meaneduc")

ntestHHI.shape

In [ ]:
ntestHHI.isnull().sum().sum()

Neste ponto são determinados os atributos analizados para a contrução do modelo

In [ ]:
XHHI = nHHI.iloc[:,0:132]
XHHI.shape

In [ ]:
YHHI = nHHI.Target

In [ ]:
XtestHHI = ntestHHI.iloc[:,0:134]
XtestHHI.shape

Em seguida serão analizados a acurácia para diversos valores de k para ser escolhido um valor para o modelo

In [ ]:
k=1
v=[]
K=[]
while k<=200:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(XHHI, YHHI)
    scores = cross_val_score(knn, XHHI, YHHI, cv=10)
    x=np.mean(scores)
    print(x)
    v.append(x)
    K.append(k)
    k+=1

In [ ]:
print(np.amax(v),np.argmax(v))

In [ ]:
vetor = pd.DataFrame(data = v)

In [ ]:

plt.plot(K, vetor)

Apesar da acurácia crescer até por volta dos 60%  nos testes realizados o modelo tende a ignorar as classes 1,2,3 e praticamente todos os resultados se tornam da classe 4. Assim será utilizado um k menor que promove uma distribuição mais gradual entre classes do modelo

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)

In [ ]:
knn.fit(XHHI,YHHI)

In [ ]:
scores = cross_val_score(knn, XHHI, YHHI, cv=10)

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
YtestHHIPred = knn.predict(XtestHHI)
YtestHHIPred

In [ ]:
predicted = pd.DataFrame(data = YtestHHIPred)
predicted[0].value_counts()

In [ ]:
predicted[0].value_counts().sort_index()

In [ ]:
predicted[0].value_counts().sort_index().plot(kind = 'bar')


Por fim, os resultados são organizados de forma que possam ser submetidos à competição

In [ ]:
result = np.vstack((testHHI["Id"],YtestHHIPred)).T

In [ ]:
x = ['Id','Target']
Result = pd.DataFrame(columns=x ,data = result)
Result

In [ ]:
Result.to_csv('resultados_Costa_Rica.csv', index=False)